Dataset with target value in json

Plaats je code voor deze delen in de stappen hieronder. Vergeet ook zeker niet de bijhorende vragen te beantwoorden.

**Opgepast:** Laat de output staan zodat het eenvoudiger is om je bekomen resultaten te interpreteren

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

In [ ]:
class BrainTumorDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        """
        Args:
            img_dir (str): Directory with all the images.
            transform (callable, optional): Optional transformations to apply to the images.
        """
        self.img_dir = img_dir
        self.transform = transform
        self.image_files = os.listdir(img_dir)  # List all image files in the directory

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Get image path
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        
        # Load image
        image = Image.open(img_path).convert("RGB")
        
        # Extract label from filename
        if "tumor" in img_name.lower():
            label = 1
        elif "no_tumor" in img_name.lower():
            label = 0
        else:
            raise ValueError(f"Cannot infer label from filename: {img_name}")
        
        # Apply transformations
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [ ]:
# Path to images
img_dir = 'brain_tumor_dataset/images'

# Data transformations for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load the custom dataset
full_dataset = BrainTumorDataset(img_dir=img_dir, transform=data_transforms['train'])

# Split into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Set the correct transform for each dataset
train_dataset.dataset.transform = data_transforms['train']  # Apply train transform to train split
val_dataset.dataset.transform = data_transforms['val']      # Apply val transform to validation split

# Create DataLoaders for training and validation
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Check the number of batches
print(f"Train loader batches: {len(train_loader)}")
print(f"Validation loader batches: {len(val_loader)}")

In [1]:
import json
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim

In [2]:
class ImageWithTargetDataset(Dataset):
    def __init__(self, img_dir, annotations_file, transform=None):
        """
        Args:
            img_dir (str): Path to the directory with the images.
            annotations_file (str): Path to the JSON file with annotations.
            transform (callable, optional): Optional transformations to apply to the images.
        """
        self.img_dir = img_dir
        self.transform = transform
        # Load annotations from the JSON file
        with open(annotations_file, 'r') as f:
            self.annotations = json.load(f)
        # Create a list of image filenames
        self.image_files = list(self.annotations.keys())

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Get the filename
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        # Load the image
        image = Image.open(img_path).convert("RGB")
        # Get the target value (people count)
        target = self.annotations[img_name]["people_count"]
        # Apply transformations
        if self.transform:
            image = self.transform(image)
        return image, target

In [3]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [4]:
img_dir = 'path_to_images'  # Replace this with the actual path to your images
annotations_file = 'annotations.json'  # Path to your JSON file with annotations

# Load the full dataset
full_dataset = ImageWithTargetDataset(img_dir=img_dir, annotations_file=annotations_file, transform=data_transforms)

# Get dataset size and print
dataset_size = len(full_dataset)
print(f"Total number of samples in the dataset: {dataset_size}")

# Split the dataset into 80% train and 20% test
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Create DataLoaders for train and test
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check batch sizes
print(f"Train loader batches: {len(train_loader)}")
print(f"Test loader batches: {len(test_loader)}")

In [5]:
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(224 * 224 * 3, 128)  # 224x224x3 = 150528 input features
        self.fc2 = nn.Linear(128, 1)  # 1 output: predicted people count

    def forward(self, x):
        x = x.view(-1, 224 * 224 * 3)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
model = RegressionModel()
criterion = nn.MSELoss()  # Mean Squared Error (MSE) for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))  # Loss expects a vector
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")